In [1]:
import operator
import pickle
import json
import numpy as np
import pandas as pd
import torch
from sklearn.metrics.cluster import normalized_mutual_info_score
from sklearn.metrics import log_loss, mean_absolute_error, mean_squared_error
from nltk.metrics.agreement import AnnotationTask
import matplotlib.pyplot as plt

In [2]:
def normalized_mi_loss(y_true, y_pred):
    return 1 - normalized_mutual_info_score(y_true, np.round(y_pred))

In [3]:
def cross_entropy_loss(y_true, y_pred):
#     return mean_squared_error(y_true, y_pred)
    try:
        return log_loss(y_true, y_pred, labels=[0,1])
    except ValueError:
        print(f'y_true {y_true}')
        print(f'y_pred {y_pred}')        

In [4]:
def jaccard_similarity(l1, l2):  #expected two lists of words or two sets of words
    l1, l2 = set(l1), set(l2)
    intersection = l1.intersection(l2)
    union = l1.union(l2)
    try:
        return float(len(intersection)) / len(union)
    except ZeroDivisionError:
        return 0

In [5]:
def load_out_and_results_files(frame_name):
    frame_out_name = f'{frame_name}_out.pkl'
    frame_res_name =  f'{frame_name}_results.csv'
    frame_out = {}
    with open(f'{frame_out_name}', 'rb') as f:
        frame_out = pickle.load(f)
    frame_res = pd.read_csv(f'{frame_res_name}')
#     print(f'{frame_res.columns}')
    return frame_out, frame_res


In [6]:
#process frame_out's information to be used by calc score
def get_organized_frame_out(frame_out):
    #get features (words)
    organized = {'feats_pos': [], 'feats_neg': [], 'N_Chunks': []}
    for i, (ri, ca) in enumerate(zip(frame_out['raw_input_list'], frame_out['conti_attr_list'])):
        #subout beginnining <s> and end </s> tokens for ['BEGIN'] and ['END']
        raw_input_i = ['[BEGIN]'if f == '<s>' else '[END]' if f == '</s>' else f for f in ri]
        attr_appearance_cutoff = 5e-2
        ca = ca.to(torch.float32)
        
        #filtering out by zeroing non-appearing features
        ca_i = torch.where(torch.abs(ca) < attr_appearance_cutoff, torch.zeros(1), ca) 
        
        #get positive and negative features
        ca_i_pos = torch.where(ca_i > 0, ca_i, torch.zeros(1))
        ca_i_neg = torch.where(ca_i < 0, ca_i, torch.zeros(1))
        
        try:
            #get idx of pos/neg identified feature
            ca_i_pos_idx = torch.nonzero(ca_i_pos).squeeze().numpy()
            ca_i_neg_idx = torch.nonzero(ca_i_neg).squeeze().numpy() 
            #don't account for empty ''s or empty arrays
            features_pos = [raw_input_i[idx] for idx in ca_i_pos_idx if ri[idx] != ''] 
            features_neg = [raw_input_i[idx] for idx in ca_i_neg_idx if ri[idx] != ''] 
        except TypeError: #TypeError: iteration over a 0-d array
            #         print(f'i: {i}')
            #         print(f'{ri}')
            #         print(f'i: {raw_input_i}')
            #         print(f'{ca}')
            #         print(f'{ca_i}')
            #         print(f'{ca_i_idx}')
            #         print(f'features frame {features_frame}')
            #         print(f'N_Chunks {N_Chunks}')
            #         print(organized)
            features_pos = []
            features_neg = []
            

        
            
        organized['feats_pos'].append(features_pos)
        organized['feats_neg'].append(features_neg)
        
        N_cs = len(features_pos) + len(features_neg)
        organized['N_Chunks'].append(N_cs)
        
#      
    frame_out.update(organized)
    return frame_out
        

In [7]:
# process frame_res to be used by calc_score
def get_organized_frame_res(frame_res):
    organized = {'yh': [], 'non_neg': [], 'non_pos': [], 'should_neg': [], 'should_pos': []}
    for x in frame_res.groupby('Input.sst2_number'):  # edit this groupby thing for different tasks
        task_answers = x[1]['Answer.taskAnswers']
        sentiments = []
        non_red_ins, non_green_ins, fiat_red_ins, fiat_green_ins = [], [], [], []
        for answer_string in task_answers:  # edit this portion below to adapt
            json_obj = json.loads(answer_string)[0]
            senti = 1 if json_obj['sentiment_radio']['1'] else 0
            sentiments.append(senti)

            if 'non_red_in' in json_obj.keys():
                non_red_ins.append([k.strip() for k in json_obj['non_red_in'].split(',')])
            else:
                non_red_ins.append([])

            if 'non_green_in' in json_obj.keys():
                non_green_ins.append([k.strip() for k in json_obj['non_green_in'].split(',')])
            else:
                non_green_ins.append([])

            if 'fiat_red_in' in json_obj.keys():
                fiat_red_ins.append([k.strip() for k in json_obj['fiat_red_in'].split(',')])
            else:
                fiat_red_ins.append([])

            if 'fiat_green_in' in json_obj.keys():
                fiat_green_ins.append([k.strip() for k in json_obj['fiat_green_in'].split(',')])
            else:
                fiat_green_ins.append([])

        organized['yh'].append(sentiments)
        organized['non_neg'].append(non_red_ins)
        organized['non_pos'].append(non_green_ins)
        organized['should_neg'].append(fiat_red_ins)
        organized['should_pos'].append(fiat_green_ins)

    # see agreement rate of turks:
    # assume 3 annotators
        
    annotation_triples = []
    for i, senti in enumerate(organized['yh'], start=1):
        a1 = ('a1', str(i), senti[0])
        a2 = ('a2', str(i), senti[1])
        a3 = ('a3', str(i), senti[2])
        annotation_triples.append(a1)
        annotation_triples.append(a2)
        annotation_triples.append(a3)      
    annotation_task = AnnotationTask(annotation_triples)
    average_ao = annotation_task.avg_Ao()

    return organized, average_ao

In [8]:
def calc_metr1_simulatability(yh, yg, ym, beta_1=1, beta_2=1):    
    loss_func = cross_entropy_loss
#     loss_func = normalized_mi_loss

    l_yhyg = loss_func(yh, yg)
    l_yhym = loss_func(yh, ym)
    print(yh, yg, l_yhyg, l_yhym)
    
    denom = beta_1 * l_yhyg + beta_2 * l_yhym + 1
    comp1 = (1/denom)
    return comp1

def metr1_wrapper(org_frame_out, org_frame_results):
    metr1s = []
    
    yhs = org_frame_results['yh']
    yms = np.round(org_frame_out['model_out_list'])
    ygs = org_frame_out['targets']

    yh_0, yh_1, yh_2 = [], [], [] #assume 3 annotaters
    for yh in yhs:
        yh_0.append(yh[0])
        yh_1.append(yh[1])
        yh_2.append(yh[2])
    
    annotations = [yh_0, yh_1, yh_2]
    for yh in annotations:
        metr1s.append(calc_metr1_simulatability(yh, ygs, yms))
#     print(np.mean(metr1s), metr1s)
    return np.mean(metr1s), metr1s
    

In [9]:
def calc_metr2_fidelity(feat_f_pos, feat_f_neg, feat_h_pos, feat_h_neg):
    jaccard_pos = jaccard_similarity(feat_f_pos, feat_h_pos)
    jaccard_neg = jaccard_similarity(feat_f_neg, feat_h_neg)
    fidelity = np.mean([jaccard_pos, jaccard_neg])
#     print(f'pos jaccard {jaccard_pos}')
#     print(f'neg jaccard {jaccard_neg}')
#     print(f'fidelity {fidelity}')
    return fidelity 

def metr2_wrapper(org_frame_out, org_frame_results):
    metr2s = []
    for ff_pos, ff_neg, fh_nn_all, fh_np_all, fh_sn_all, fh_sp_all\
                            in zip(org_frame_out['feats_pos'], 
                               org_frame_out['feats_neg'], 
                               org_frame_results['non_neg'],
                               org_frame_results['non_pos'],
                               org_frame_results['should_neg'],
                               org_frame_results['should_pos']):
        metr2_annos = []
        for fh_nn, fh_np, fh_sn, fh_sp\
                            in zip(fh_nn_all, fh_np_all, fh_sn_all, fh_sp_all):
            fh_neg = set(ff_neg).difference(fh_nn).union(fh_sn)
            fh_pos = set(ff_pos).difference(fh_nn).union(fh_sp)
            
            metr2_an = calc_metr2_fidelity(ff_pos, ff_neg, fh_pos, fh_neg)
            metr2_annos.append(metr2_an)
        metr2s.append(metr2_annos)
    metr2s = np.array(metr2s)
    metr2s_average = metr2s.mean(axis=1).mean(axis=0)
    return metr2s_average, metr2s


In [10]:
def calc_metr3_complexity(N_c):
    if N_c == 0:
        return 0
    else:
        return 1/(np.log(N_c)+1)

def metr3_wrapper(org_frame_out, org_frame_results):
    metr3s = []
    N_Chunks = org_frame_out['N_Chunks']
    for N_c in N_Chunks:
        metr3 = calc_metr3_complexity(N_c)
        metr3s.append(metr3)
    metr3s_average = np.mean(metr3s)
#     print(metr3s_average, metr3s)
    return metr3s_average, metr3s


In [11]:
def calc_m1m2m3(org_frame_out, org_frame_results):
    m1, metric1s = metr1_wrapper(org_frame_out, org_frame_results)
    m2, metric2s = metr2_wrapper(org_frame_out, org_frame_results)
    m3, metric3s = metr3_wrapper(org_frame_out, org_frame_results)
    return m1, m2, m3

In [12]:
def score_wrapper(frame_names=['deep_lift', 'guided_backprop', 'input_x_gradients', 'integrated_gradients', 'kernel_shap', 'lime']):
    avg_aos = []
    m1s, m2s, m3s = [], [], []
    for frame_name in frame_names:
        print(f'Framework processed: {frame_name}')
        frame_out, frame_results = load_out_and_results_files(frame_name)
        org_frame_results, avg_ao = get_organized_frame_res(frame_results)
        avg_aos.append(avg_ao)
        org_frame_out = get_organized_frame_out(frame_out)    
        m1, m2, m3 = calc_m1m2m3(org_frame_out, org_frame_results)
        m1s.append(m1)
        m2s.append(m2)
        m3s.append(m3)
        print(f'm1: {m1:4f}, m2: {m2:4f}, m3: {m3:4f}')
    print(f'average average agreement {np.mean(avg_aos):.2f}')
    return frame_names, m1s, m2s, m3s

In [13]:
frame_names, m1s, m2s, m3s = score_wrapper(['deep_lift', 'guided_backprop', 'input_x_gradients', 'integrated_gradients', 'kernel_shap', 'lime'])

Framework processed: deep_lift
[1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0] [1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0] 7.598642750520578 6.907867222622364
[1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1] [1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0] 13.815606509655895 14.506398029502712
[1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0] [1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0

In [14]:
def see_1_3_alpha(frame_names, m1s, m2s, m3s):
    for f, m1, m2, m3 in zip(frame_names, m1s, m2s, m3s):
        m1_13 = m1 * (1/3)
        m2_13 = m2 * (1/3)
        m3_13 = m3 * (1/3)
        score_13 = m1_13 + m2_13 + m3_13
        print(F"\\textbf{{f}} & {m1_13:.4f} & {m2_13:.4f} & {m3_13:.4f} & {score_13:.4f} \\\\ {f}")

see_1_3_alpha(frame_names, m1s, m2s, m3s)


\textbf{f} & 0.0471 & 0.2430 & 0.1067 & 0.3969 \\ deep_lift
\textbf{f} & 0.0376 & 0.1784 & 0.0805 & 0.2965 \\ guided_backprop
\textbf{f} & 0.0504 & 0.2462 & 0.1067 & 0.4033 \\ input_x_gradients
\textbf{f} & 0.0761 & 0.1698 & 0.0810 & 0.3269 \\ integrated_gradients
\textbf{f} & 0.0291 & 0.2555 & 0.1063 & 0.3909 \\ kernel_shap
\textbf{f} & 0.0443 & 0.1726 & 0.0839 & 0.3008 \\ lime


In [15]:
def generate_alpha_combos():
    import itertools
    alphas = np.arange(0, 11, step=1, dtype=np.uint8)
    all_combos = [(a/10,b/10,c/10) for (a,b,c) in itertools.product(alphas, alphas, alphas) if np.sum([a,b,c]) == 10]
    #reverse all_combos
    all_combos.reverse()
    return all_combos

In [16]:
alpha_combos = generate_alpha_combos()

In [17]:
def apply_alphas(frame_names, m1s, m2s, m3s, alpha_combos):
    scores_list = [] #indexed by the alpha combinations
    for combo in alpha_combos:
        a1, a2, a3 = combo[0], combo[1], combo[2]
        scores_for_fs = []
        for f, m1, m2, m3 in zip(frame_names, m1s, m2s, m3s):
            a1m1, a2m2, a3m3 = a1 * m1, a2 * m2, a3 * m3
            score = a1m1 + a2m2 + a3m3
            scores_for_fs.append(score)      
        scores_list.append([frame_names, scores_for_fs])
    return scores_list


In [18]:
scores_list = apply_alphas(frame_names, m1s, m2s, m3s, alpha_combos)

In [19]:
# def normalize_score_list(scores_list):
#     normalized_scores_list = []
#     from sklearn.preprocessing import normalize
#     def min_max_norm(scores):
#         norm_scores = (scores-np.min(scores))/(np.max(scores)-np.min(scores))
#         return norm_scores
    
#     for (frame_names, scores) in scores_list:
#         scores = np.array(scores)
# #         normalized_scores = normalize([scores])[0]
# #         normalized_scores = scores / np.sum(scores)
#         normalized_scores = min_max_norm(scores)
# #         print(f'after normalizing: {normalized_scores}\n-------------------------------------')
#         normalized_scores_list.append([frame_names, normalized_scores])
#     return normalized_scores_list

# scores_list = normalize_score_list(scores_list)  

In [20]:
def get_frame_scores(scores_list, alpha_combos):
    frame_names = scores_list[0][0]
    frame_scores = {}
    for i in range(len(frame_names)):
        frame_name = frame_names[i]
        frame_scores[frame_name] = [scores_list[j][1][i] for j in range(len(scores_list))]
    
   
    
    #sort frames based on average scores
    average_scores_dict = {}
    for frame, frame_scores_list in frame_scores.items():
        print(f'Average score for {frame}: {np.mean(frame_scores_list)}, std: {np.std(frame_scores_list)}')
        average_scores_dict[frame] = np.mean(frame_scores_list)
    sorted_average_scores_list =[[k, v] for k, v in sorted(average_scores_dict.items(), 
                                                          key=lambda item: item[1], reverse=True)]
    print(f'sorted average scores {sorted_average_scores_list}')

    ordered_frame_names = [k for [k,v] in sorted_average_scores_list]
    #sort based on top frame
    top_frame_name = ordered_frame_names[0]
    print(f'top_frame_name {top_frame_name}')
    sorted_idxs = np.argsort(frame_scores[top_frame_name])
    sorted_frame_scores = {}
    for frame in frame_names:
        sorted_frame_scores[frame] = np.array(frame_scores[frame])[sorted_idxs]
    
    permuted_alphas = np.array(np.array(alpha_combos)[sorted_idxs])
    a1_idx = np.where(permuted_alphas[:,0]==1)[0][0]
    a2_idx = np.where(permuted_alphas[:,1]==1)[0][0]
    a3_idx = np.where(permuted_alphas[:,2]==1)[0][0]
    
    
    return sorted_frame_scores, a1_idx, a2_idx, a3_idx, ordered_frame_names 

In [21]:
frame_scores, a1_idx, a2_idx, a3_idx, ordered_frame_names = get_frame_scores(scores_list, alpha_combos)

Average score for deep_lift: 0.39687235533397397, std: 0.14022648230580823
Average score for guided_backprop: 0.2965005900629861, std: 0.10074505842962399
Average score for input_x_gradients: 0.40326281885313825, std: 0.1407494709018223
Average score for integrated_gradients: 0.3269174591340922, std: 0.07363006464432033
Average score for kernel_shap: 0.3908783973524882, std: 0.1607269017494883
Average score for lime: 0.30083882738174844, std: 0.09172773480152567
sorted average scores [['input_x_gradients', 0.40326281885313825], ['deep_lift', 0.39687235533397397], ['kernel_shap', 0.3908783973524882], ['integrated_gradients', 0.3269174591340922], ['lime', 0.30083882738174844], ['guided_backprop', 0.2965005900629861]]
top_frame_name input_x_gradients


In [22]:
def output_graph_info(frame_scores, a1_idx, a2_idx, a3_idx, ordered_frame_names, task_name='SST2'):
    out_dict = {
        'task_name': task_name,
        'frame_scores': frame_scores, 
        'a1_idx': a1_idx, 
        'a2_idx': a2_idx,
        'a3_idx': a3_idx,
        'ordered_frame_names': ordered_frame_names
    }
    out_file_name = f'{task_name}_graph.pkl'
    with open(out_file_name, 'wb') as f:
        pickle.dump(out_dict, f)
    return out_dict

In [23]:
output_graph_info(frame_scores, a1_idx, a2_idx, a3_idx, ordered_frame_names)

{'task_name': 'SST2',
 'frame_scores': {'deep_lift': array([0.14135922, 0.15924377, 0.17712833, 0.19501288, 0.20012861,
         0.21289744, 0.21801316, 0.23078199, 0.23589771, 0.24866654,
         0.25378227, 0.25889799, 0.2665511 , 0.27166682, 0.27678255,
         0.28443565, 0.28955138, 0.2946671 , 0.30232021, 0.30743593,
         0.31255166, 0.32020476, 0.31766738, 0.32532048, 0.33043621,
         0.33555193, 0.34320504, 0.34832076, 0.35343649, 0.36108959,
         0.36620532, 0.37132104, 0.37643677, 0.38408987, 0.3892056 ,
         0.39432132, 0.40197443, 0.40709015, 0.41220587, 0.4249747 ,
         0.43009043, 0.43520615, 0.44285926, 0.44797498, 0.45309071,
         0.46585954, 0.47097526, 0.48374409, 0.48885981, 0.49397554,
         0.50674437, 0.51186009, 0.52462892, 0.52974465, 0.5476292 ,
         0.55274493, 0.56551376, 0.57062948, 0.58851403, 0.60639859,
         0.61151431, 0.62939887, 0.64728342, 0.6702837 , 0.68816825,
         0.72905309]),
  'guided_backprop': array([0